In [14]:
# 코랩 환경에서 셀레니움 라이브러리 설치
# ! pip install selenium

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# 크롬 옵션 설정 (헤드리스 모드)
chrome_options = Options()
# chrome_options.add_argument('--headless')  # GUI 표시 X
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

In [16]:
from datetime import datetime

include = input('기사 제목에 포함시킬 내용을 입력하세요: ')

while True:
    start_date = input('시작 날짜를 입력하세요(예시: 20xx-xx-xx): ')
    end_date = input('종료 날짜를 입력하세요(예시: 20xx-xx-xx): ')
    
    try:
        start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
        end_date_obj = datetime.strptime(end_date, "%Y-%m-%d")
        
        if end_date_obj >= start_date_obj:
            break
        else:
            print("종료 날짜가 시작 날짜보다 빠를 수 없습니다. 다시 입력해주세요.")
    
    except ValueError:
        print("날짜 형식이 올바르지 않습니다. 다시 입력해주세요.")


start_date = list(map(int, start_date.split('-')))
end_date = list(map(int, end_date.split('-')))

In [17]:
# 날짜 선택 함수
def select_date(start_date, end_date):
    global driver
    time.sleep(3)

    # 시작 날짜
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{start_date[0]-1989}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{start_date[1]}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{start_date[2]}]/a').click()

    # 종료 날짜 클릭
    driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[1]/span[3]/a').click()

    # 종료 날짜
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{end_date[0]-1989}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{end_date[1]}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{end_date[2]}]/a').click()

    # 적용 클릭
    driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[3]/button').click()

In [18]:
# 크롬 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

driver.get(url=f'https://search.naver.com/search.naver?where=news&query={include}')

# 최신순 클릭
driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[1]/a[2]').click()
# 옵션 클릭
driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[2]/a').click()
driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]').click()

# 날짜 선택
select_date(start_date, end_date)

last_height = driver.execute_script("return document.body.scrollHeight")  # 현재 페이지의 전체 높이

# 스크롤링을 위한 반복문
while True:
    # 스크롤을 최하단으로 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 로드 시간 대기

    # 새로운 페이지 높이를 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 새로운 페이지 높이가 기존 높이와 같으면, 더 이상 스크롤 불가능
    if new_height == last_height:
        break
    last_height = new_height  # 마지막 높이 갱신

press = [element.get_attribute("textContent") for element in driver.find_elements(By.CLASS_NAME, "info.press")] # 언론사
title = [element.get_attribute("title") for element in driver.find_elements(By.CLASS_NAME, "news_tit")] # 기사 제목
link = [element.get_attribute("href") for element in driver.find_elements(By.CLASS_NAME, "news_tit")]   # 기사 링크

driver.quit()   # 드라이버 종료

# 수집한 데이터 출력
for t, p, l in zip(title, press, link):
    print(f"언론사: {p} | 제목: {t} | 링크: {l}")

언론사: 아유경제 | 제목: [아유경제_부동산] 정밀도 높인 '국가 지오이드모델' 공개 | 링크: http://www.areyou.co.kr/news/articleView.html?idxno=86131
언론사: 매일건설신문 | 제목: 국토지리정보원, 정밀도 높인 ‘국가 지오이드모델’ 공개 | 링크: http://mcnews.co.kr/82488
언론사: 스카이데일리 | 제목: 국토지리정보원 ‘정밀도 향상된 국가 지오이드모델 KNGeoid24’ 공개 | 링크: https://m.skyedaily.com/news_view.html?ID=249636
언론사: 뉴스캠프 | 제목: 국토교통부, 정밀도 향상된 국가 지오이드모델(KNGeoid24) 제공 시작 | 링크: https://www.newscammp.co.kr/news/articleView.html?idxno=2966
언론사: 내일신문 | 제목: 최신 ‘국가 지오이드모델’ 공개 | 링크: https://www.naeil.com/news/read/525765?ref=naver
언론사: 헬로티 | 제목: 건설, 토목 현장 정밀도 높일 '24 국가 지오이드모델' 공개 | 링크: https://www.hellot.net/news/article.html?no=94430
언론사: MBC | 제목: 한반도 해발고도 측량 더 정확해진다 | 링크: https://imnews.imbc.com/news/2024/econo/article/6646761_36452.html
언론사: 매일안전신문 | 제목: 국토지리정보원, 정밀도 높인 '2024 국가 지오이드모델' 공개 | 링크: https://idsn.co.kr/news/view/1065576586597524
언론사: 국토일보 | 제목: 정밀도 높인 '2024 국가 지오이드모델' 공개… GNSS 위성 측량 지원 | 링크: http://www.ikld.kr/news/articleView.html?idxno=301782
언론사: 마켓뉴스 | 제목: 정밀도 높인 '국가 지오이드모델' 공개...

In [19]:
import pandas as pd

# 데이터프레임 생성
df = pd.DataFrame({
    "언론사": press,
    "기사 제목": title,
    "링크": link
}).drop_duplicates(subset=['언론사','기사 제목']).reset_index(drop=True)    # 중복 기사 제거
df.index = range(1, len(df) + 1)
df.rename_axis('순번', axis='index', inplace=True)
# df.to_csv('article_dataframe.csv', encoding='utf-8-sig')  # csv 파일로 저장
df.to_excel(f'{include} 언론보도 현황.xlsx')   # xlsx 파일로 저장

# 데이터프레임 출력
display(df)

,언론사,기사 제목,링크
순번,,,
1,아유경제,[아유경제_부동산] 정밀도 높인 '국가 지오이드모델' 공개,http://www.areyou.co.kr/news/articleView.html?...
2,매일건설신문,"국토지리정보원, 정밀도 높인 ‘국가 지오이드모델’ 공개",http://mcnews.co.kr/82488
3,스카이데일리,국토지리정보원 ‘정밀도 향상된 국가 지오이드모델 KNGeoid24’ 공개,https://m.skyedaily.com/news_view.html?ID=249636
4,뉴스캠프,"국토교통부, 정밀도 향상된 국가 지오이드모델(KNGeoid24) 제공 시작",https://www.newscammp.co.kr/news/articleView.h...
5,내일신문,최신 ‘국가 지오이드모델’ 공개,https://www.naeil.com/news/read/525765?ref=naver
6,헬로티,"건설, 토목 현장 정밀도 높일 '24 국가 지오이드모델' 공개",https://www.hellot.net/news/article.html?no=94430
7,MBC,한반도 해발고도 측량 더 정확해진다,https://imnews.imbc.com/news/2024/econo/articl...
8,매일안전신문,"국토지리정보원, 정밀도 높인 '2024 국가 지오이드모델' 공개",https://idsn.co.kr/news/view/1065576586597524
9,국토일보,정밀도 높인 '2024 국가 지오이드모델' 공개… GNSS 위성 측량 지원,http://www.ikld.kr/news/articleView.html?idxno...


In [20]:
# from google.colab import files

# 파일 다운로드
# files.download(f'{include} 언론보도 현황.csv')   # csv 파일 다운로드
# files.download(f'{include} 언론보도 현황.xlsx')    # xlsx 파일 다운로드